# 50 persons
## electrodes : 8, 16, 32, 64

In [1]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from xgboost import XGBClassifier

2023-11-30 16:24:15.733698: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 16:24:15.765957: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-30 16:24:15.766005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-30 16:24:15.767081: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-30 16:24:15.772859: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 16:24:15.773387: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
pip install mne

Note: you may need to restart the kernel to use updated packages.


In [2]:
no_of_patients=50

In [3]:
train=sorted(glob('files3/*.edf'))
train=train[:no_of_patients]
train

['files3/S001R05.edf',
 'files3/S002R05.edf',
 'files3/S003R05.edf',
 'files3/S004R05.edf',
 'files3/S005R05.edf',
 'files3/S006R05.edf',
 'files3/S007R05.edf',
 'files3/S008R05.edf',
 'files3/S009R05.edf',
 'files3/S010R05.edf',
 'files3/S011R05.edf',
 'files3/S012R05.edf',
 'files3/S013R05.edf',
 'files3/S014R05.edf',
 'files3/S015R05.edf',
 'files3/S016R05.edf',
 'files3/S017R05.edf',
 'files3/S018R05.edf',
 'files3/S019R05.edf',
 'files3/S020R05.edf',
 'files3/S021R05.edf',
 'files3/S022R05.edf',
 'files3/S023R05.edf',
 'files3/S024R05.edf',
 'files3/S025R05.edf',
 'files3/S026R05.edf',
 'files3/S027R05.edf',
 'files3/S028R05.edf',
 'files3/S029R05.edf',
 'files3/S030R05.edf',
 'files3/S031R05.edf',
 'files3/S032R05.edf',
 'files3/S033R05.edf',
 'files3/S034R05.edf',
 'files3/S035R05.edf',
 'files3/S036R05.edf',
 'files3/S037R05.edf',
 'files3/S038R05.edf',
 'files3/S039R05.edf',
 'files3/S040R05.edf',
 'files3/S041R05.edf',
 'files3/S042R05.edf',
 'files3/S043R05.edf',
 'files3/S0

In [4]:
def read_data(i,train_split,valid_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
    
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample(frac=1)
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    idx3=int(valid_split*(len(eeg_df2)))
    idx4=int(valid_split*(len(eeg_df2)))+1
    eeg_df3=eeg_df2.iloc[:idx3]
    eeg_df4=eeg_df2.iloc[idx4:]
    return eeg_df1,eeg_df3,eeg_df4,len(eeg_df1),len(eeg_df3),len(eeg_df4)

    

In [5]:
%%capture
xtemp1=[]
xtemp2=[]
xtemp3=[]
ytemp1=[]
ytemp2=[]
ytemp3=[]
for i in range(no_of_patients):
    xtr,xte,xval,ytr,yte,yval=read_data(train[i],0.8,0.5) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    xtemp3.append(xval)
    ytemp1.append(ytr)
    ytemp2.append(yte)
    ytemp3.append(yval)

In [6]:
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)
xvalid=pd.concat([xtemp3[i] for i in range(0,len(xtemp3))],ignore_index=True)

In [7]:
ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        
yvalid=[]
for i in range(len(ytemp3)):
    for j in range(ytemp3[i-1]):
        yvalid.append(i)     

In [8]:
len(xtrain),len(xtest),len(ytrain),len(ytest)

(600000, 74950, 600000, 74950)

In [9]:
print(xtest.iloc[:,-1].values)

[ 1.01e-04  1.30e-05 -1.50e-05 ... -1.20e-05 -7.00e-06  1.50e-05]


In [10]:
display(xtrain)

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000010,0.000020,0.000007,-0.000018,0.000000,0.000004,0.000060,0.000004,0.000012,0.000006,...,-0.000006,0.000030,0.000025,0.000022,0.000013,-0.000009,0.000020,0.000024,0.000089,0.000068
1,0.000010,0.000050,0.000049,0.000022,0.000038,0.000031,0.000084,0.000025,0.000043,0.000049,...,0.000001,0.000044,0.000031,0.000027,0.000025,0.000012,0.000030,0.000025,0.000083,0.000065
2,0.000017,0.000055,0.000059,0.000030,0.000037,0.000025,0.000079,0.000030,0.000048,0.000053,...,0.000006,0.000048,0.000036,0.000033,0.000034,0.000034,0.000041,0.000034,0.000092,0.000078
3,0.000021,0.000063,0.000066,0.000035,0.000040,0.000029,0.000076,0.000035,0.000055,0.000065,...,0.000029,0.000048,0.000041,0.000050,0.000059,0.000063,0.000045,0.000047,0.000121,0.000086
4,0.000027,0.000072,0.000079,0.000048,0.000052,0.000042,0.000091,0.000043,0.000060,0.000076,...,0.000035,0.000029,0.000022,0.000042,0.000062,0.000071,0.000037,0.000051,0.000143,0.000090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,-0.000016,-0.000025,-0.000018,-0.000031,-0.000033,-0.000026,-0.000028,0.000004,-0.000003,0.000007,...,0.000021,0.000016,0.000020,0.000014,0.000021,0.000011,0.000018,0.000014,0.000019,0.000027
599996,-0.000005,-0.000013,-0.000003,-0.000014,-0.000016,-0.000008,-0.000009,0.000018,0.000002,0.000016,...,0.000014,-0.000002,-0.000002,0.000001,0.000010,0.000003,-0.000001,-0.000005,0.000005,0.000013
599997,-0.000003,-0.000006,-0.000002,-0.000011,-0.000008,-0.000004,-0.000003,0.000011,-0.000007,0.000011,...,-0.000003,-0.000024,-0.000021,-0.000022,-0.000008,-0.000015,-0.000020,-0.000019,-0.000008,0.000004
599998,-0.000005,-0.000015,-0.000001,-0.000014,-0.000021,-0.000011,-0.000008,-0.000008,-0.000010,0.000006,...,0.000034,0.000008,0.000013,0.000018,0.000034,0.000036,0.000020,0.000037,0.000054,0.000072


In [11]:
def scale_dataset(dataframe):
    x=dataframe.iloc[:,:-1].values
    y=dataframe.iloc[:,-1].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 0-8

In [12]:
xtrain8=xtrain.iloc[:,:8]
xvalid8=xvalid.iloc[:,:8]
xtest8=xtest.iloc[:,:8]

In [13]:
xtrain8['id']=ytrain
xtest8['id']=ytest
xvalid8['id']=yvalid
display(xtrain)

/tmp/ipykernel_20950/394288547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain8['id']=ytrain
/tmp/ipykernel_20950/394288547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest8['id']=ytest
/tmp/ipykernel_20950/394288547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000010,0.000020,0.000007,-0.000018,0.000000,0.000004,0.000060,0.000004,0.000012,0.000006,...,-0.000006,0.000030,0.000025,0.000022,0.000013,-0.000009,0.000020,0.000024,0.000089,0.000068
1,0.000010,0.000050,0.000049,0.000022,0.000038,0.000031,0.000084,0.000025,0.000043,0.000049,...,0.000001,0.000044,0.000031,0.000027,0.000025,0.000012,0.000030,0.000025,0.000083,0.000065
2,0.000017,0.000055,0.000059,0.000030,0.000037,0.000025,0.000079,0.000030,0.000048,0.000053,...,0.000006,0.000048,0.000036,0.000033,0.000034,0.000034,0.000041,0.000034,0.000092,0.000078
3,0.000021,0.000063,0.000066,0.000035,0.000040,0.000029,0.000076,0.000035,0.000055,0.000065,...,0.000029,0.000048,0.000041,0.000050,0.000059,0.000063,0.000045,0.000047,0.000121,0.000086
4,0.000027,0.000072,0.000079,0.000048,0.000052,0.000042,0.000091,0.000043,0.000060,0.000076,...,0.000035,0.000029,0.000022,0.000042,0.000062,0.000071,0.000037,0.000051,0.000143,0.000090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,-0.000016,-0.000025,-0.000018,-0.000031,-0.000033,-0.000026,-0.000028,0.000004,-0.000003,0.000007,...,0.000021,0.000016,0.000020,0.000014,0.000021,0.000011,0.000018,0.000014,0.000019,0.000027
599996,-0.000005,-0.000013,-0.000003,-0.000014,-0.000016,-0.000008,-0.000009,0.000018,0.000002,0.000016,...,0.000014,-0.000002,-0.000002,0.000001,0.000010,0.000003,-0.000001,-0.000005,0.000005,0.000013
599997,-0.000003,-0.000006,-0.000002,-0.000011,-0.000008,-0.000004,-0.000003,0.000011,-0.000007,0.000011,...,-0.000003,-0.000024,-0.000021,-0.000022,-0.000008,-0.000015,-0.000020,-0.000019,-0.000008,0.000004
599998,-0.000005,-0.000015,-0.000001,-0.000014,-0.000021,-0.000011,-0.000008,-0.000008,-0.000010,0.000006,...,0.000034,0.000008,0.000013,0.000018,0.000034,0.000036,0.000020,0.000037,0.000054,0.000072


In [14]:
x8,y8=scale_dataset(xtrain8)
xt8,yt8=scale_dataset(xtest8)
xv8,yv8=scale_dataset(xvalid8)

In [15]:
model=XGBClassifier(n_estimators=500)
model.fit(x8,y8,early_stopping_rounds=10, eval_set=[(xv8, yv8)])
y_pred=model.predict(xt8)
print(classification_report(yt8,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.62314
[1]	validation_0-mlogloss:3.50683
[2]	validation_0-mlogloss:3.42044
[3]	validation_0-mlogloss:3.35586
[4]	validation_0-mlogloss:3.30245
[5]	validation_0-mlogloss:3.25946
[6]	validation_0-mlogloss:3.22365
[7]	validation_0-mlogloss:3.19035
[8]	validation_0-mlogloss:3.16422
[9]	validation_0-mlogloss:3.13997
[10]	validation_0-mlogloss:3.11837
[11]	validation_0-mlogloss:3.09870
[12]	validation_0-mlogloss:3.08202
[13]	validation_0-mlogloss:3.06454
[14]	validation_0-mlogloss:3.04789
[15]	validation_0-mlogloss:3.03457
[16]	validation_0-mlogloss:3.01966
[17]	validation_0-mlogloss:3.00399
[18]	validation_0-mlogloss:2.99344
[19]	validation_0-mlogloss:2.98219
[20]	validation_0-mlogloss:2.96891
[21]	validation_0-mlogloss:2.95982
[22]	validation_0-mlogloss:2.94947
[23]	validation_0-mlogloss:2.93916
[24]	validation_0-mlogloss:2.92707
[25]	validation_0-mlogloss:2.91699
[26]	validation_0-mlogloss:2.91043
[27]	validation_0-mlogloss:2.90048
[28]	validation_0-mlogloss:2.8

In [16]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(8, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(50, activation = 'linear', name = "L6"),
    ]
)

In [17]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x8,y8,epochs=10,validation_data=(xv8,yv8)
)

Epoch 1/10
18750/18750 [==============================] - 13s 669us/step - loss: 2.3515 - val_loss: 2.4842
Epoch 2/10
18750/18750 [==============================] - 14s 734us/step - loss: 1.8619 - val_loss: 2.4132
Epoch 3/10
18750/18750 [==============================] - 15s 819us/step - loss: 1.7733 - val_loss: 2.4211
Epoch 4/10
18750/18750 [==============================] - 16s 865us/step - loss: 1.6820 - val_loss: 2.3772
Epoch 5/10
18750/18750 [==============================] - 39s 2ms/step - loss: 1.6014 - val_loss: 2.4270
Epoch 6/10
18750/18750 [==============================] - 17s 892us/step - loss: 1.5581 - val_loss: 2.4179
Epoch 7/10
18750/18750 [==============================] - 15s 800us/step - loss: 1.5285 - val_loss: 2.3913
Epoch 8/10
18750/18750 [==============================] - 16s 830us/step - loss: 1.5089 - val_loss: 2.4746
Epoch 9/10
18750/18750 [==============================] - 16s 879us/step - loss: 1.4928 - val_loss: 2.4768
Epoch 10/10
18750/18750 [==============

In [18]:
y_pred_classes = np.argmax(tf.nn.softmax(model.predict(xt8)).numpy(),axis=1)
print(classification_report(yt8,y_pred_classes))

2343/2343 [==============================] - 1s 334us/step
              precision    recall  f1-score   support

           0       0.13      0.15      0.14      1499
           1       0.11      0.09      0.10      1499
           2       0.29      0.24      0.26      1499
           3       0.55      0.27      0.36      1499
           4       0.67      0.73      0.70      1499
           5       0.38      0.42      0.40      1499
           6       0.25      0.57      0.35      1499
           7       0.47      0.39      0.43      1499
           8       0.56      0.58      0.57      1499
           9       0.54      0.46      0.50      1499
          10       0.31      0.37      0.33      1499
          11       0.33      0.39      0.36      1499
          12       0.38      0.29      0.33      1499
          13       0.20      0.23      0.21      1499
          14       0.22      0.07      0.11      1499
          15       0.44      0.46      0.45      1499
          16       0.1

## 0-16

In [16]:
xtrain16=xtrain.iloc[:,:16]
xtest16=xtest.iloc[:,:16]
xvalid16=xvalid.iloc[:,:16]

In [17]:
xtrain16['id']=ytrain
xtest16['id']=ytest
xvalid16['id']=yvalid
display(xtrain)

/tmp/ipykernel_20950/4039718790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain16['id']=ytrain
/tmp/ipykernel_20950/4039718790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest16['id']=ytest
/tmp/ipykernel_20950/4039718790.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000010,0.000020,0.000007,-0.000018,0.000000,0.000004,0.000060,0.000004,0.000012,0.000006,...,-0.000006,0.000030,0.000025,0.000022,0.000013,-0.000009,0.000020,0.000024,0.000089,0.000068
1,0.000010,0.000050,0.000049,0.000022,0.000038,0.000031,0.000084,0.000025,0.000043,0.000049,...,0.000001,0.000044,0.000031,0.000027,0.000025,0.000012,0.000030,0.000025,0.000083,0.000065
2,0.000017,0.000055,0.000059,0.000030,0.000037,0.000025,0.000079,0.000030,0.000048,0.000053,...,0.000006,0.000048,0.000036,0.000033,0.000034,0.000034,0.000041,0.000034,0.000092,0.000078
3,0.000021,0.000063,0.000066,0.000035,0.000040,0.000029,0.000076,0.000035,0.000055,0.000065,...,0.000029,0.000048,0.000041,0.000050,0.000059,0.000063,0.000045,0.000047,0.000121,0.000086
4,0.000027,0.000072,0.000079,0.000048,0.000052,0.000042,0.000091,0.000043,0.000060,0.000076,...,0.000035,0.000029,0.000022,0.000042,0.000062,0.000071,0.000037,0.000051,0.000143,0.000090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,-0.000016,-0.000025,-0.000018,-0.000031,-0.000033,-0.000026,-0.000028,0.000004,-0.000003,0.000007,...,0.000021,0.000016,0.000020,0.000014,0.000021,0.000011,0.000018,0.000014,0.000019,0.000027
599996,-0.000005,-0.000013,-0.000003,-0.000014,-0.000016,-0.000008,-0.000009,0.000018,0.000002,0.000016,...,0.000014,-0.000002,-0.000002,0.000001,0.000010,0.000003,-0.000001,-0.000005,0.000005,0.000013
599997,-0.000003,-0.000006,-0.000002,-0.000011,-0.000008,-0.000004,-0.000003,0.000011,-0.000007,0.000011,...,-0.000003,-0.000024,-0.000021,-0.000022,-0.000008,-0.000015,-0.000020,-0.000019,-0.000008,0.000004
599998,-0.000005,-0.000015,-0.000001,-0.000014,-0.000021,-0.000011,-0.000008,-0.000008,-0.000010,0.000006,...,0.000034,0.000008,0.000013,0.000018,0.000034,0.000036,0.000020,0.000037,0.000054,0.000072


In [18]:
x16,y16=scale_dataset(xtrain16)
xt16,yt16=scale_dataset(xtest16)
xv16,yv16=scale_dataset(xvalid16)

In [19]:
model2=XGBClassifier(n_estimators=500)
model2.fit(x16,y16,early_stopping_rounds=10, eval_set=[(xv16, yv16)])
y_pred=model2.predict(xt16)
print(classification_report(yt16,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.54984
[1]	validation_0-mlogloss:3.39544
[2]	validation_0-mlogloss:3.28727
[3]	validation_0-mlogloss:3.19921
[4]	validation_0-mlogloss:3.13020
[5]	validation_0-mlogloss:3.07111
[6]	validation_0-mlogloss:3.01865
[7]	validation_0-mlogloss:2.97252
[8]	validation_0-mlogloss:2.93814
[9]	validation_0-mlogloss:2.89854
[10]	validation_0-mlogloss:2.86595
[11]	validation_0-mlogloss:2.83768
[12]	validation_0-mlogloss:2.80821
[13]	validation_0-mlogloss:2.78426
[14]	validation_0-mlogloss:2.76289
[15]	validation_0-mlogloss:2.74434
[16]	validation_0-mlogloss:2.72400
[17]	validation_0-mlogloss:2.70546
[18]	validation_0-mlogloss:2.68571
[19]	validation_0-mlogloss:2.66621
[20]	validation_0-mlogloss:2.64900
[21]	validation_0-mlogloss:2.63569
[22]	validation_0-mlogloss:2.61768
[23]	validation_0-mlogloss:2.60113
[24]	validation_0-mlogloss:2.58854
[25]	validation_0-mlogloss:2.57486
[26]	validation_0-mlogloss:2.56252
[27]	validation_0-mlogloss:2.54736
[28]	validation_0-mlogloss:2.5

In [22]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model1 = Sequential(
    [
        Dense(16, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(50, activation = 'linear', name = "L6"),
    ]
)

In [23]:
model1.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model1.fit(
    x16,y16,epochs=10,validation_data=(xv16,yv16)
)

Epoch 1/10
18750/18750 [==============================] - 534s 28ms/step - loss: 1.8313 - val_loss: 2.0093
Epoch 2/10
18750/18750 [==============================] - 20s 1ms/step - loss: 1.0621 - val_loss: 2.1111
Epoch 3/10
18750/18750 [==============================] - 18s 943us/step - loss: 0.8859 - val_loss: 2.0982
Epoch 4/10
18750/18750 [==============================] - 15s 820us/step - loss: 0.8101 - val_loss: 2.1486
Epoch 5/10
18750/18750 [==============================] - 15s 808us/step - loss: 0.7654 - val_loss: 2.1447
Epoch 6/10
18750/18750 [==============================] - 15s 795us/step - loss: 0.7342 - val_loss: 2.3253
Epoch 7/10
18750/18750 [==============================] - 15s 777us/step - loss: 0.7100 - val_loss: 2.2375
Epoch 8/10
18750/18750 [==============================] - 15s 795us/step - loss: 0.6922 - val_loss: 2.2986
Epoch 9/10
18750/18750 [==============================] - 15s 809us/step - loss: 0.6766 - val_loss: 2.3375
Epoch 10/10
18750/18750 [==============

In [24]:
y_pred_classes = np.argmax(tf.nn.softmax(model1.predict(xt16)).numpy(),axis=1)
print(classification_report(yt16,y_pred_classes))

2343/2343 [==============================] - 1s 333us/step
              precision    recall  f1-score   support

           0       0.26      0.30      0.28      1499
           1       0.30      0.17      0.22      1499
           2       0.33      0.38      0.35      1499
           3       0.52      0.28      0.37      1499
           4       0.79      0.85      0.82      1499
           5       0.61      0.43      0.50      1499
           6       0.59      0.69      0.64      1499
           7       0.90      0.53      0.67      1499
           8       0.44      0.58      0.50      1499
           9       0.64      0.62      0.63      1499
          10       0.55      0.37      0.44      1499
          11       0.66      0.68      0.67      1499
          12       0.47      0.62      0.54      1499
          13       0.33      0.41      0.36      1499
          14       0.26      0.27      0.27      1499
          15       0.87      0.62      0.72      1499
          16       0.6

## 0-32

In [20]:
xtrain32=xtrain.iloc[:,:32]
xtest32=xtest.iloc[:,:32]
xvalid32=xvalid.iloc[:,:32]

In [21]:
xtrain32['id']=ytrain
xtest32['id']=ytest
xvalid32['id']=yvalid

/tmp/ipykernel_20950/1675347936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain32['id']=ytrain
/tmp/ipykernel_20950/1675347936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest32['id']=ytest
/tmp/ipykernel_20950/1675347936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [22]:
x32,y32=scale_dataset(xtrain32)
xt32,yt32=scale_dataset(xtest32)
xv32,yv32=scale_dataset(xvalid32)

In [23]:
model3=XGBClassifier(n_estimators=500)
model3.fit(x32,y32,early_stopping_rounds=10, eval_set=[(xv32, yv32)])
y_pred=model3.predict(xt32)
print(classification_report(yt32,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.34618
[1]	validation_0-mlogloss:3.11471
[2]	validation_0-mlogloss:2.95737
[3]	validation_0-mlogloss:2.83498
[4]	validation_0-mlogloss:2.73736
[5]	validation_0-mlogloss:2.65904
[6]	validation_0-mlogloss:2.58793
[7]	validation_0-mlogloss:2.52424
[8]	validation_0-mlogloss:2.46481
[9]	validation_0-mlogloss:2.41477
[10]	validation_0-mlogloss:2.37046
[11]	validation_0-mlogloss:2.33119
[12]	validation_0-mlogloss:2.29122
[13]	validation_0-mlogloss:2.25572
[14]	validation_0-mlogloss:2.22221
[15]	validation_0-mlogloss:2.19537
[16]	validation_0-mlogloss:2.17056
[17]	validation_0-mlogloss:2.14280
[18]	validation_0-mlogloss:2.11735
[19]	validation_0-mlogloss:2.09372
[20]	validation_0-mlogloss:2.06853
[21]	validation_0-mlogloss:2.04530
[22]	validation_0-mlogloss:2.02768
[23]	validation_0-mlogloss:2.00976
[24]	validation_0-mlogloss:1.98859
[25]	validation_0-mlogloss:1.97224
[26]	validation_0-mlogloss:1.95498
[27]	validation_0-mlogloss:1.94095
[28]	validation_0-mlogloss:1.9

In [28]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model2 = Sequential(
    [
        Dense(32, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(50, activation = 'linear', name = "L6"),
    ]
)

In [29]:
model2.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model2.fit(
    x32,y32,epochs=10,validation_data=(xv32,yv32)
)

Epoch 1/10
18750/18750 [==============================] - 16s 847us/step - loss: 1.1451 - val_loss: 1.6261
Epoch 2/10
18750/18750 [==============================] - 16s 859us/step - loss: 0.4678 - val_loss: 1.6150
Epoch 3/10
18750/18750 [==============================] - 17s 881us/step - loss: 0.3352 - val_loss: 1.4612
Epoch 4/10
18750/18750 [==============================] - 17s 902us/step - loss: 0.2695 - val_loss: 1.4778
Epoch 5/10
18750/18750 [==============================] - 16s 872us/step - loss: 0.2351 - val_loss: 1.4624
Epoch 6/10
18750/18750 [==============================] - 17s 885us/step - loss: 0.2118 - val_loss: 1.3247
Epoch 7/10
18750/18750 [==============================] - 19s 989us/step - loss: 0.1952 - val_loss: 1.3887
Epoch 8/10
18750/18750 [==============================] - 16s 879us/step - loss: 0.1812 - val_loss: 1.4315
Epoch 9/10
18750/18750 [==============================] - 17s 893us/step - loss: 0.1709 - val_loss: 1.3067
Epoch 10/10
18750/18750 [============

In [30]:
y_pred_classes = np.argmax(tf.nn.softmax(model2.predict(xt32)).numpy(),axis=1)
print(classification_report(yt32,y_pred_classes))

2343/2343 [==============================] - 1s 438us/step
              precision    recall  f1-score   support

           0       0.71      0.67      0.69      1499
           1       0.56      0.64      0.59      1499
           2       0.68      0.83      0.75      1499
           3       0.73      0.26      0.39      1499
           4       0.98      0.95      0.97      1499
           5       0.97      0.88      0.92      1499
           6       0.95      0.91      0.93      1499
           7       0.97      0.79      0.87      1499
           8       0.55      0.80      0.65      1499
           9       0.84      0.74      0.79      1499
          10       0.96      0.88      0.92      1499
          11       0.93      0.98      0.95      1499
          12       0.71      0.88      0.78      1499
          13       0.78      0.80      0.79      1499
          14       0.61      0.71      0.66      1499
          15       0.94      0.96      0.95      1499
          16       0.8

## 0-64

In [24]:
xtrain['id']=ytrain
xtest['id']=ytest
xvalid['id']=yvalid

In [25]:
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)
xv,yv=scale_dataset(xvalid)

In [26]:
model4=XGBClassifier(n_estimators=500)
model4.fit(x,y,early_stopping_rounds=10, eval_set=[(xv, yv)])
y_pred=model4.predict(xt)
print(classification_report(yt,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.02927
[1]	validation_0-mlogloss:2.74625
[2]	validation_0-mlogloss:2.55927
[3]	validation_0-mlogloss:2.41789
[4]	validation_0-mlogloss:2.29925
[5]	validation_0-mlogloss:2.20087
[6]	validation_0-mlogloss:2.11668
[7]	validation_0-mlogloss:2.04918
[8]	validation_0-mlogloss:1.98173
[9]	validation_0-mlogloss:1.92439
[10]	validation_0-mlogloss:1.86914
[11]	validation_0-mlogloss:1.82111
[12]	validation_0-mlogloss:1.78006
[13]	validation_0-mlogloss:1.74138
[14]	validation_0-mlogloss:1.70463
[15]	validation_0-mlogloss:1.67397
[16]	validation_0-mlogloss:1.64273
[17]	validation_0-mlogloss:1.61474
[18]	validation_0-mlogloss:1.58396
[19]	validation_0-mlogloss:1.55625
[20]	validation_0-mlogloss:1.53280
[21]	validation_0-mlogloss:1.50739
[22]	validation_0-mlogloss:1.48700
[23]	validation_0-mlogloss:1.46718
[24]	validation_0-mlogloss:1.44793
[25]	validation_0-mlogloss:1.42842
[26]	validation_0-mlogloss:1.40921
[27]	validation_0-mlogloss:1.39221
[28]	validation_0-mlogloss:1.3

In [33]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model3 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(50, activation = 'linear', name = "L6"),
    ]
)

In [34]:
model3.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model3.fit(
    x,y,epochs=10,validation_data=(xv,yv)
)

Epoch 1/10
18750/18750 [==============================] - 18s 951us/step - loss: 0.7520 - val_loss: 1.6733
Epoch 2/10
18750/18750 [==============================] - 18s 950us/step - loss: 0.2907 - val_loss: 1.7687
Epoch 3/10
18750/18750 [==============================] - 17s 931us/step - loss: 0.2129 - val_loss: 1.9209
Epoch 4/10
18750/18750 [==============================] - 287s 15ms/step - loss: 0.1717 - val_loss: 1.7423
Epoch 5/10
18750/18750 [==============================] - 23s 1ms/step - loss: 0.1481 - val_loss: 1.7941
Epoch 6/10
18750/18750 [==============================] - 18s 953us/step - loss: 0.1294 - val_loss: 1.9329
Epoch 7/10
18750/18750 [==============================] - 16s 862us/step - loss: 0.1156 - val_loss: 1.8122
Epoch 8/10
18750/18750 [==============================] - 16s 869us/step - loss: 0.1056 - val_loss: 1.5906
Epoch 9/10
18750/18750 [==============================] - 16s 861us/step - loss: 0.0973 - val_loss: 1.5869
Epoch 10/10
18750/18750 [==============

In [35]:
y_pred_classes = np.argmax(tf.nn.softmax(model3.predict(xt)).numpy(),axis=1)
print(classification_report(yt,y_pred_classes))

2343/2343 [==============================] - 1s 334us/step
              precision    recall  f1-score   support

           0       0.78      0.85      0.81      1499
           1       0.83      0.90      0.86      1499
           2       0.89      0.79      0.84      1499
           3       0.93      0.82      0.87      1499
           4       0.95      1.00      0.97      1499
           5       0.94      0.94      0.94      1499
           6       0.93      0.99      0.96      1499
           7       0.99      0.81      0.89      1499
           8       0.72      0.89      0.80      1499
           9       0.82      0.74      0.78      1499
          10       0.77      0.90      0.83      1499
          11       0.94      0.96      0.95      1499
          12       0.91      0.94      0.92      1499
          13       0.65      0.92      0.76      1499
          14       0.65      0.74      0.69      1499
          15       0.96      0.94      0.95      1499
          16       0.7